#Modélisation
1-Feature selection
2-Finetuning/Cross Validation

### Chargement des données

In [1]:
import pandas as pd
from src.data_eng.prepare_db import *

full_dataset = load_dataset("bank-additional-full.csv")
print("Le dataset contient", len(full_dataset), "lignes")

Le dataset contient 41188 lignes


### Définition des types de variables
- Features en entrée par type

In [2]:
features = full_dataset.columns[:-1]
continuous_features = [col for col in features if full_dataset[col].dtype != 'object']
print("Variables continues:", continuous_features)
categorical_features = [col for col in features if full_dataset[col].dtype == 'object']
print("Variables categoriques:", categorical_features)
# On vérifie qu'on oublie rien (des dates ou autres)
print("Toutes les variables sont prises en compte:",
      (set(categorical_features).union(continuous_features)).difference(set(features)))

Variables continues: ['age', 'duration', 'campaign', 'pdays', 'previous', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed']
Variables categoriques: ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome']
Toutes les variables sont prises en compte: set()


- Réencodage de la cible

In [3]:
target_column = "y"
full_dataset[target_column] = full_dataset[target_column].map({"yes": 1, "no": 0})


### On encode les variables catégoriques avec du one-hot encoding

In [4]:
from src.modelling.cross_validation import *
full_dataset_encoded = pd.get_dummies(full_dataset, columns = categorical_features, drop_first=True)

### On divise le dataset en train/test

In [5]:
train, test = split_dataset(full_dataset_encoded, 0.6)# fonction custom mais on peut utiliser sklearn
train_targets = train[target_column]
test_targets = test[target_column]
train = train.drop(columns=["y"])
test = test.drop(columns = ["y"])

### Selection de feature.
- Modèle de base pour avoir l'importance des features
- Sélection itérative en retirant les features les moins importantes
- Evaluation sur la base de test

In [ ]:
from xgboost import XGBClassifier
from src.modelling.cross_validation import feature_selection
results_selection = feature_selection(XGBClassifier, train, train_targets, test,test_targets, metric=f1_score)

/Users/philippe.stepniewski/projets/Dojo_Banking/venv/lib/python3.9/site-packages/sklearn/base.py:402: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/philippe.stepniewski/projets/Dojo_Banking/venv/lib/python3.9/site-packages/sklearn/base.py:402: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/philippe.stepniewski/projets/Dojo_Banking/venv/lib/python3.9/site-packages/sklearn/base.py:402: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


Thresh=0.000, n=53, Accuracy: 59.44%


/Users/philippe.stepniewski/projets/Dojo_Banking/venv/lib/python3.9/site-packages/sklearn/base.py:402: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/philippe.stepniewski/projets/Dojo_Banking/venv/lib/python3.9/site-packages/sklearn/base.py:402: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


Thresh=0.000, n=53, Accuracy: 59.44%


/Users/philippe.stepniewski/projets/Dojo_Banking/venv/lib/python3.9/site-packages/sklearn/base.py:402: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/philippe.stepniewski/projets/Dojo_Banking/venv/lib/python3.9/site-packages/sklearn/base.py:402: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


...

Thresh=0.007, n=41, Accuracy: 58.57%


/Users/philippe.stepniewski/projets/Dojo_Banking/venv/lib/python3.9/site-packages/sklearn/base.py:402: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/philippe.stepniewski/projets/Dojo_Banking/venv/lib/python3.9/site-packages/sklearn/base.py:402: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


Thresh=0.007, n=40, Accuracy: 58.69%


/Users/philippe.stepniewski/projets/Dojo_Banking/venv/lib/python3.9/site-packages/sklearn/base.py:402: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/philippe.stepniewski/projets/Dojo_Banking/venv/lib/python3.9/site-packages/sklearn/base.py:402: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


Thresh=0.007, n=39, Accuracy: 59.05%


- Affichage des performances en test en fonction du nombre de features en entrée

In [ ]:
results_selection = pd.DataFrame(results_selection)
plt.plot(results_selection["number of features"],results_selection["metric"])
print(results_selection[results_selection["number of features"] == 13].selected_features)